# Imports Part

In [37]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report, accuracy_score


# Loading Labelled data

In [19]:
# Loading Labelled data
data = pd.read_csv("C:/Users/47819/OneDrive/Desktop/STT 890/steelcase/labelled_floorplan_dataset.csv")

def image_path(key):
    return f"Total_Images/{key}/{key}_Top.png"

data['image_path'] = data['key'].apply(image_path)
data

image_paths = data['image_path'].values
labels = data['label'].values

image_paths = data['image_path'].values
labels = data['label'].values

# Define dataset class
class FloorplanDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets and data loaders
dataset = FloorplanDataset(image_paths, labels, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Load the ResNet50 model pre-trained on image data

In [20]:
# 加载预训练的ResNet50模型
resnet = models.resnet50(pretrained=True)
resnet.fc = nn.Identity() 


C:\Users\47819\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\47819\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# ResNet50 Extract Features

In [24]:
# extract features by ResNet50
def extract_features(data_loader, model):
    model.eval()
    features = []
    labels = []

    with torch.no_grad():
        for inputs, labels_batch in data_loader:
            outputs = model(inputs)
            features.append(outputs)
            labels.append(labels_batch)

    features = torch.cat(features).numpy()
    labels = torch.cat(labels).numpy()
    return features, labels

# Extract training set features
features, labels = extract_features(train_loader, resnet)


# Split the data set into training set and test set

In [25]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


# Classifier Part

In [29]:
# K-NN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
f1_knn = f1_score(y_test, y_pred_knn, average='macro')

# XGBoost
y_train_xgb = np.where(y_train == -1, 0, 1)
y_test_xgb = np.where(y_test == -1, 0, 1)
xgb = XGBClassifier()
xgb.fit(X_train, y_train_xgb)
y_pred_xgb = xgb.predict(X_test)
f1_xgb = f1_score(y_test_xgb, y_pred_xgb, average='macro')

print(f'K-NN F1 Score: {f1_knn}')
print(f'XGBoost F1 Score: {f1_xgb}')


K-NN F1 Score: 0.7633603990871942
XGBoost F1 Score: 0.7533875338753389


In [36]:
print(classification_report(y_test, y_pred_knn))
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn)}")

              precision    recall  f1-score   support

        -1.0       0.93      0.97      0.95       560
         1.0       0.69      0.49      0.58        77

    accuracy                           0.91       637
   macro avg       0.81      0.73      0.76       637
weighted avg       0.90      0.91      0.91       637

Accuracy: 0.9120879120879121


In [35]:
print(classification_report(y_test_xgb, y_pred_xgb))
print(f"Accuracy: {accuracy_score(y_test_xgb, y_pred_xgb)}")

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       560
           1       0.71      0.45      0.56        77

    accuracy                           0.91       637
   macro avg       0.82      0.71      0.75       637
weighted avg       0.90      0.91      0.90       637

Accuracy: 0.9120879120879121
